# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [29]:
import random
from string import punctuation
from collections import Counter
import numpy as np
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [30]:
with open('2ch_corpus.txt', 'r', encoding='utf-8') as f:
    dvach = f.readlines()
    test_corpora = random.sample(dvach, 50)

    for i in test_corpora:
        dvach.remove(i)

    dvach = '\n'.join(dvach)



with open('lenta.txt', 'r', encoding='utf-8') as f:
    news = f.read()


In [31]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11719282
Длина 2 - 11536552


In [32]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [33]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858107
Длина корпуса новостных текстов в токенах -  1505789


In [34]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [35]:
vocab_dvach.most_common(10)

[('и', 55861),
 ('в', 48834),
 ('не', 46577),
 ('на', 29647),
 ('что', 26649),
 ('я', 21724),
 ('а', 21302),
 ('с', 21071),
 ('это', 17721),
 ('ты', 15459)]

In [36]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

In [37]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [38]:
def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(1/len(norm_dvach)))
    
    return np.exp(prob)

In [39]:
phrase = "владимир путин подписал указ о введении новых налогов на богатых россиян"
if compute_joint_proba(phrase, probas_dvach) > compute_joint_proba(phrase, probas_news):
    print("Текст похож на токсичные посты")
else:
    print("Текст похож на новостные тексты")

Текст похож на новостные тексты


In [40]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]


In [41]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))

trigrams_dvach = Counter()
trigrams_news = Counter()

for sentence in sentences_dvach:
    trigrams_dvach.update(ngrammer(sentence, 3))

for sentence in sentences_news:
    trigrams_news.update(ngrammer(sentence, 3))

In [42]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(text) + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        # small value for unk words
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

def compute_joint_proba_trigram(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = ['<start>', '<start>'] + normalize(text) + ['<end>']
    for ngram in ngrammer(tokens, n=3):
        word1, word2, word3 = ngram.split()
        bigram = f"{word1} {word2}"
        trigram = f"{word1} {word2} {word3}"
        if bigram in bigram_counts and trigram in trigram_counts and bigram_counts[bigram] > 0:
            prob += np.log(trigram_counts[trigram] / bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    return np.exp(prob)

In [43]:
phrase = "Технические возможности бесполезного российского судна не позволили разгрузить его у терминала"
if compute_joint_proba_trigram(phrase, bigrams_dvach, trigrams_dvach) > compute_joint_proba_trigram(phrase, bigrams_news, trigrams_news):
    print("Текст похож на токсичные посты")
else:
    print("Текст похож на новостные тексты")

Текст похож на новостные тексты


In [44]:
# матрица слова на слова (инициализируем нулями)
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                          len(unigrams_dvach)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_dvach = list(unigrams_dvach)
print(len(id2word_dvach))
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
print(len(id2bigram_dvach))
bigram2id_dvach = {ngram:i for i, ngram in enumerate(id2bigram_dvach)}

id2trigram_dvach = list(trigrams_dvach)
print(len(id2trigram_dvach))
trigram2id_dvach = {ngram:i for i, ngram in enumerate(id2trigram_dvach)}

110835
494948
692060


In [45]:
id2bigram_dvach[0]

'<start> <start>'

In [46]:
# заполняем матрицу
for ngram in trigrams_dvach:
    bigram, word2 = " ".join(ngram.split()[:2]), ngram.split()[2]
    try:
        # на пересечение двух слов ставим вероятность встретить второе после первого
        matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word2]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])
    except Exception as e:
        print(f"Error processing ngram \"{ngram}\": {e}")
matrix_dvach = csc_matrix(matrix_dvach)

In [47]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        probs = matrix[current_idx].toarray()[0]
        probs_sum = probs.sum()
        if probs_sum == 0:
            # fallback: end the sentence
            text.append('<end>')
            break
        probs = probs / probs_sum  # normalize
        chosen = np.random.choice(matrix.shape[1], p=probs)
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            break
        # update current_idx to the new bigram
        prev_bigram = id2bigram[current_idx].split()
        new_bigram = f"{prev_bigram[1]} {id2word[chosen]}"
        current_idx = bigram2id.get(new_bigram, bigram2id['<start> <start>'])
    return ' '.join(text)

In [48]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, id2bigram_dvach, bigram2id_dvach).replace('<end>', '\n'))

блжад с базой данных пользователей 



In [49]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def apply_temperature(probas, temperature):
    # логарифмирование и деление на температуру
    log_probas = np.log(np.maximum(probas, 1e-10))  
    adjusted_log_probas = log_probas / temperature
    # чтобы получить честные вероятности, нужно применить софтмакс
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

def compute_joint_proba_triplet(text, word_probas, bigram_probas):
    prob = 0
    tokens = normalize(text)
    for i in range(len(tokens) - 2):
        word1, word2, word3 = tokens[i], tokens[i + 1], tokens[i + 2]
        bigram = f"{word1} {word2}"
        if bigram in bigram_probas and word3 in word_probas:
            prob += np.log(bigram_probas[bigram]) + np.log(word_probas[word3])
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens) - 2

In [50]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'
perplexity(*compute_joint_proba(phrase, probas_dvach))

np.float64(17732.353112215213)

In [51]:
perplexity(*compute_joint_proba_triplet(phrase, probas_dvach, bigrams_dvach))

np.float64(3951.4400591275294)

In [53]:
ps = []
for sent in sent_tokenize('\n'.join(test_corpora)):
    prob, N = compute_joint_proba_triplet(sent, probas_dvach, bigrams_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))

np.mean(ps)

np.float64(3168.1353134089277)

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.